In [ ]:
# ============================================
# Track Meet Planning Under Uncertainty
# ============================================

import math
import random

# Seed for reproducibility
random.seed(2025)

# -------------------------------
# Problem Setup
# -------------------------------

# Suppose we have 3 athletes with base abilities (lower is better)
athletes = {
    "A1": 10.8,  # consistent performer
    "A2": 11.0,  # slightly slower
    "A3": 10.6   # fastest but erratic
}

# Each athlete has a probability distribution over their race time
# We'll model this as a Gaussian with different standard deviations
performance_profiles = {
    "A1": {"mean": 10.8, "std": 0.05},
    "A2": {"mean": 11.0, "std": 0.04},
    "A3": {"mean": 10.6, "std": 0.15}
}

# -------------------------------
# Simulate Probabilistic Outcomes
# -------------------------------

def simulate_race(athlete_id, n=1000):
    profile = performance_profiles[athlete_id]
    samples = [round(random.gauss(profile["mean"], profile["std"]), 3) for _ in range(n)]
    return samples

# Simulate 1000 races for each athlete
samples_A1 = simulate_race("A1")
samples_A2 = simulate_race("A2")
samples_A3 = simulate_race("A3")

# -------------------------------
# Compute Probabilities
# -------------------------------

# What's the probability that A3 beats A1?
wins_A3_over_A1 = sum(1 for a3, a1 in zip(samples_A3, samples_A1) if a3 < a1) / len(samples_A3)

# What's the probability that A1 finishes under 10.75s?
fast_A1 = sum(1 for t in samples_A1 if t < 10.75) / len(samples_A1)

# -------------------------------
# Decision-Theoretic Planning
# -------------------------------

# Suppose we must choose one athlete to anchor the relay team.
# We define utility as:
# - High utility for finishing under 10.75s
# - Medium utility for finishing under 11.0s
# - Low utility otherwise

def expected_utility(samples):
    utility = 0
    for t in samples:
        if t < 10.75:
            utility += 10
        elif t < 11.0:
            utility += 5
        else:
            utility += 1
    return utility / len(samples)

eu_A1 = expected_utility(samples_A1)
eu_A2 = expected_utility(samples_A2)
eu_A3 = expected_utility(samples_A3)

# -------------------------------
# Output Summary
# -------------------------------

print("=== Probabilistic Performance Summary ===")
print(f"P(A3 beats A1): {wins_A3_over_A1:.2f}")
print(f"P(A1 < 10.75s): {fast_A1:.2f}")

print("\n=== Expected Utilities for Relay Anchor ===")
print(f"A1: {eu_A1:.2f}")
print(f"A2: {eu_A2:.2f}")
print(f"A3: {eu_A3:.2f}")

# Decision: Choose athlete with highest expected utility
best_choice = max(["A1", "A2", "A3"], key=lambda a: expected_utility(simulate_race(a)))
print(f"\nRecommended anchor: {best_choice}")